<hr style="border:0.2px solid black"> </hr>

<figure>
  <IMG SRC="img/ntnu-norwegian-university-of-science-and-technology-vector-logo.png" WIDTH=250 ALIGN="right">
</figure>

**<ins>Course:</ins>** TVM4174 - Hydroinformatics for Smart Water Systems

# <ins>Example 4c:</ins> Simple Examples that EPANET can't do
    
*Developed by David B. Steffelbauer*

<hr style="border:0.2px solid black"> </hr>

This notebook is slightly adapted from an example in the WNTR documentation [$\rightarrow$link](https://wntr.readthedocs.io/en/stable/morph.html)
    
## Skeletonization

In [ ]:
%matplotlib inline
import wntr
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
wn = wntr.network.WaterNetworkModel('data/L-Town.inp')
wntr.graphics.plot_network(wn, node_size=4)
wn.describe()

In [ ]:
wntr.morph.skeletonize?

In [ ]:
skel_wn = wntr.morph.skeletonize(wn, 1.0)


wntr.graphics.plot_network(skel_wn, node_size=4)
skel_wn.describe()

In [ ]:
d_orig = pd.Series(dict(nx.degree(wn.get_graph())))
d_skel = pd.Series(dict(nx.degree(skel_wn.get_graph())))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
wntr.graphics.plot_network(wn, node_size=20, title='Original', node_attribute=d_orig, node_colorbar_label='Degree', ax=ax1)
wntr.graphics.plot_network(skel_wn, node_size=20, title='Skeletonized', node_attribute=d_skel, node_colorbar_label='Degree', ax=ax2);

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,5))
d_orig.hist(bins=np.arange(0, 7)-.5, ax=ax1, label='Original')
d_skel.hist(bins=np.arange(0, 7)-.5, ax=ax1, label='Skeletonized')
plt.xlabel('node degree', fontsize=16)
plt.ylabel('# nodes', fontsize=16)
plt.legend(fontsize=14);

In [ ]:
sim = wntr.sim.EpanetSimulator(wn)
results_original = sim.run_sim()

In [ ]:
sim = wntr.sim.EpanetSimulator(skel_wn)
# %timeit results_skel = sim.run_sim()
results_skel = sim.run_sim()

In [ ]:
skel_junctions = skel_wn.junction_name_list
pressure_orig = results_original.node['pressure'].loc[:,skel_junctions]
pressure_skel = results_skel.node['pressure'].loc[:,skel_junctions]
pressure_diff = (abs(pressure_orig - pressure_skel)/pressure_orig)*100
pressure_diff.index = pressure_diff.index/3600 # convert time to hours

In [ ]:
confidence = 0.9

alpha = 1-confidence

lower = pressure_diff.quantile(alpha/2, axis=1)
median = pressure_diff.quantile(0.50, axis=1)
upper = pressure_diff.quantile(1 - alpha/2, axis=1)

In [ ]:
fig = plt.figure()
ax = median.plot()
poly = ax.fill_between(lower.index, lower, upper, color='b', alpha=0.2)
text = ax.set_xlabel('Time (hr)')
text = ax.set_ylabel('Percent change in pressure (%)')